#### <div style= "font-family: Cambria font-weight:bold letter-spacing: 0px color:black font-size:150% text-align:leftpadding:3.0px background: #cceeff border-bottom: 8px solid #004466" > TABLE OF CONTENTS<br><div>  
* [IMPORTS](#1)
* [INTRODUCTION](#2)
    * [CONFIGURATION](#2.1)
    * [CONFIGURATION PARAMETERS](#2.2)    
    * [DATASET COLUMNS](#2.3)
* [PREPROCESSING](#3)
* [MODEL TRAINING](#4)      
* [PLANNED WAY FORWARD](#5) 

In [2]:
%%time 

# Installing select libraries:-
from gc import collect
from warnings import filterwarnings
filterwarnings('ignore')
from IPython.display import display_html, clear_output

# ! python -m pip install --no-index \
# --find-links=/kaggle/input/packageinstallation \
# -r /kaggle/input/packageinstallation/requirements.txt -q
# clear_output()

import xgboost as xgb, lightgbm as lgb, catboost as cb, sklearn as sk
print(f"---> XGBoost = {xgb.__version__} | LightGBM = {lgb.__version__} | Catboost = {cb.__version__}")
print(f"---> Sklearn = {sk.__version__}\n\n")
collect()

OSError: dlopen(/Users/kausik/.local/share/virtualenvs/BharatIntern-hi361nWe/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <AF2B35E2-56EE-3669-88A6-BBD75577D81F> /Users/kausik/.local/share/virtualenvs/BharatIntern-hi361nWe/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [2]:
%%time

# General library imports:-
from copy import deepcopy
import pandas as pd
import numpy as np
import re
from scipy.stats import mode, kstest, normaltest, shapiro, anderson, jarque_bera
from collections import Counter
from itertools import product
from colorama import Fore, Style, init
from warnings import filterwarnings
filterwarnings('ignore')
import joblib
import os

from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap as LCM
%matplotlib inline

from pprint import pprint
from functools import partial

print()
collect()
clear_output()

CPU times: user 243 ms, sys: 22.2 ms, total: 265 ms
Wall time: 337 ms


In [3]:
%%time 

# Importing model and pipeline specifics:-
# importing category_encoders:- OrdinalEncoder, OneHotEncoder
from category_encoders import OrdinalEncoder, OneHotEncoder

# Pipeline specifics:-
from sklearn.preprocessing import (RobustScaler, 
                                   MinMaxScaler, 
                                   StandardScaler, 
                                   FunctionTransformer as FT,
                                   PowerTransformer,
                                  )
from sklearn.impute import SimpleImputer as SI
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score, cross_val_predict
                                    )
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# ML Model training:-
from sklearn.metrics import mean_squared_error as mse, make_scorer
from xgboost import DMatrix, XGBRegressor as XGBR
from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR, Pool
from sklearn.ensemble import (HistGradientBoostingRegressor as HGBR, 
                              RandomForestRegressor as RFR,
                              ExtraTreesRegressor as ETR,
                              GradientBoostingRegressor as GBR,
                             )
from sklearn.linear_model import Ridge as RIDGER, Lasso as LASSOR
from sklearn.neighbors import KNeighborsRegressor as KNNR

# Ensemble and tuning:-
import optuna
from optuna import Trial, trial, create_study
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler, CmaEsSampler
optuna.logging.set_verbosity = optuna.logging.CRITICAL

clear_output()
print()
collect()


CPU times: user 1.45 s, sys: 120 ms, total: 1.57 s
Wall time: 1.93 s


In [4]:
%%time 

# Setting rc parameters in seaborn for plots and graphs- 
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,  
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,        
        })

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL) 

# Making sklearn pipeline outputs as dataframe:-
from sklearn import set_config 
set_config(transform_output = "pandas")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print()
collect()



CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 132 ms


| Version<br>Number | Version Details | CV score| Single/ Ensemble|Public LB Score|
| :-: | --- | :-: | :-: |:-:|
| **V1** |* Used the FE component from high scoring public notebook <br> * Attached my FE dataset <br> * Used my training pipeline <br> * Optuna ensemble, 10x1 ML model CV <br> * Blended with good public work |0.11325|Optuna <br> Ensemble|0.0766|

In [5]:
%%time

# Configuration class:-
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Some parameters may be unused here as this is a general configuration class
    """
    
    # Data preparation:-   
    version_nb         = 1
    test_req           = "N"
    test_sample_frac   = 0.025
    gpu_switch         = "OFF" 
    state              = 42
    target             = 'SalePrice'
    data_path          = f"/kaggle/input/housepricestraintestfe"
    path               = f"/kaggle/input/house-prices-advanced-regression-techniques"
    tgt_path           = f"/kaggle/input/ames-housing-dataset"
    
    ftre_imp_req       = "N"
    scl_method         = "Robust"
        
    # Model Training:- 
    pstprcs_oof        = "N"
    pstprcs_train      = "N"
    pstprcs_test       = "N"
    ML                 = "Y"
    
    pseudo_lbl_req     = "N"
    pseudolbl_up       = 1e10
    pseudolbl_low      = 0.05
    n_splits           = 3 if test_req == "Y" else 5
    n_repeats          = 1 
    nbrnd_erly_stp     = 65
    mdlcv_mthd         = 'RKF'
    
    # Ensemble:-    
    ensemble_req       = "Y"
    hill_climb_req     = "N"
    optuna_req         = "Y"
    LAD_req            = "N"
    enscv_mthd         = "RKF"
    metric_obj         = 'minimize'
    ntrials            = 10 if test_req == "Y" else 200
    
    # Blending with public work:-     
    blend_wgt          = [0.20, 0.10, 0.10, 0.60]
    
    # Global variables for plotting:-
    grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--', 
                           'color': 'lightgrey', 'linewidth': 0.75}
    title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'}


print()
PrintColor(f"--> Configuration done!\n")
collect()


--> Configuration done!

CPU times: user 130 ms, sys: 750 µs, total: 131 ms
Wall time: 129 ms


In [6]:
%time 

class Preprocessor():
    """
    This class aims to do the below-
    1. Read the datasets
    2. Check information and description
    3. Check unique values and nulls
    4. Collate starting features 
    """
    
    def __init__(self):
        self.train = pd.read_csv(os.path.join(CFG.data_path,"XYtrain.csv"), index_col = "Unnamed: 0")
        self.test  = \
        pd.read_csv(os.path.join(CFG.data_path ,"Xtest.csv"), index_col = "Id").\
        drop(["Unnamed: 0"], axis=1, errors = "ignore")
        
        self.target   = CFG.target 
        self.test_req = CFG.test_req
        
        self.ytrain   = pd.read_csv(os.path.join(CFG.tgt_path, "AmesHousing.csv"), 
                                    usecols = [CFG.target]
                                   )
        self.ytrain   = np.log1p(self.ytrain)
        self.sub_fl   = pd.read_csv(os.path.join(CFG.path, "sample_submission.csv"))
        
        self.train = self.train.drop(columns = [self.target], axis=1, errors = "ignore")
        PrintColor(f"Data shapes - train-test = {self.train.shape} {self.test.shape}")
                  
        PrintColor(f"\nTrain set head", color = Fore.CYAN)
        display(self.train.head(5).style.format(precision = 3))
        PrintColor(f"\nTest set head", color = Fore.CYAN)
        display(self.test.head(5).style.format(precision = 3))
        
        self.orig_ftre = [c for c in self.test.columns if c not in ['id', "Id", "Source", "Label"]]
        
    def _RenameCols(self):
        "This method renames columns to prevent issues in models"
        self.col_mapper = pd.DataFrame.from_dict({"OrigCols": self.orig_ftre})
        self.col_mapper["ColLbl"] = [f"Col{i}" for i in range(len(self.orig_ftre))]
        
        for df in [self.train, self.test]:
            df.columns = self.col_mapper["ColLbl"]
            
        self.strt_ftre = list(self.col_mapper["ColLbl"].values)
        return self
                              
    def _CollateInfoDesc(self):
        PrintColor(f"\n{'-'*20} Information and description {'-'*20}\n", color = Fore.MAGENTA)

        # Creating dataset information and description:
        for lbl, df in {'Train': self.train, 'Test': self.test}.items():
            PrintColor(f"\n{lbl} description\n")
            display(df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).\
                    transpose().\
                    drop(columns = ['count'], errors = 'ignore').\
                    drop([CFG.target], axis=0, errors = 'ignore').\
                    style.format(formatter = '{:,.2f}').\
                    background_gradient(cmap = 'Blues')
                   )

            PrintColor(f"\n{lbl} information\n")
            display(df.info())
            collect()
        return self
    
    @staticmethod
    def ReduceMem(df: pd.DataFrame):
        "This method reduces memory for numeric columns in the dataframe"
        
        numerics  = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        start_mem = df.memory_usage().sum() / 1024**2
        
        for col in df.columns:
            col_type = df[col].dtypes
            
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()

                if "int" in str(col_type):
                    if c_min >= np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min >= np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min >= np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min >= np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min >= np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    if c_min >= np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)  

        end_mem = df.memory_usage().sum() / 1024**2
    
        PrintColor(f"Start - end memory:- {start_mem:5.2f} - {end_mem:5.2f} Mb")
        return df
    
    def _CollateUnqNull(self): 
        # Dislaying the unique values across train-test-original:-
        PrintColor(f"\nUnique and null values\n")
        _ = pd.concat([self.train[self.strt_ftre].nunique(), 
                       self.test[self.strt_ftre].nunique(), 
                       self.train[self.strt_ftre].isna().sum(axis=0),
                       self.test[self.strt_ftre].isna().sum(axis=0),
                      ], 
                      axis=1)
        _.columns = ['Train_Nunq', 'Test_Nunq', 'Train_Nulls', 'Test_Nulls']

        display(_.T.style.background_gradient(cmap = 'Blues', axis=1).\
                format(formatter = '{:,.0f}')
               )
            
        return self
    
       
    def DoPreprocessing(self):
        print()
        self._RenameCols()
        self._CollateInfoDesc()
        self._CollateUnqNull() 
        print()
        
        for df in [self.train, self.test]:
            df = df.replace([np.inf, -1*np.inf], np.NaN )
        
        return self.train, self.ytrain, self.test 
          
collect()
print()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [7]:
%%time 

pp = Preprocessor()
Xtrain, ytrain, Xtest = pp.DoPreprocessing()

print()
collect()

Data shapes - train-test = (2930, 356) (1459, 356)

Train set head


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,Age,Renovate,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_None,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,Utilities_None,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_CBlock,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,BsmtQual_TA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC


Test set head


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,Age,Renovate,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_None,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,Utilities_None,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_CBlock,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,BsmtQual_TA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC



-------------------- Information and description --------------------


Train description



...


Test information

<class 'pandas.core.frame.DataFrame'>
Index: 1459 entries, 1461 to 2919
Columns: 356 entries, Col0 to Col355
dtypes: float64(309), int64(47)
memory usage: 4.0 MB


None


Unique and null values



ColLbl,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,Col57,Col58,Col59,Col60,Col61,Col62,Col63,Col64,Col65,Col66,Col67,Col68,Col69,Col70,Col71,Col72,Col73,Col74,Col75,Col76,Col77,Col78,Col79,Col80,Col81,Col82,Col83,Col84,Col85,Col86,Col87,Col88,Col89,Col90,Col91,Col92,Col93,Col94,Col95,Col96,Col97,Col98,Col99,Col100,Col101,Col102,Col103,Col104,Col105,Col106,Col107,Col108,Col109,Col110,Col111,Col112,Col113,Col114,Col115,Col116,Col117,Col118,Col119,Col120,Col121,Col122,Col123,Col124,Col125,Col126,Col127,Col128,Col129,Col130,Col131,Col132,Col133,Col134,Col135,Col136,Col137,Col138,Col139,Col140,Col141,Col142,Col143,Col144,Col145,Col146,Col147,Col148,Col149,Col150,Col151,Col152,Col153,Col154,Col155,Col156,Col157,Col158,Col159,Col160,Col161,Col162,Col163,Col164,Col165,Col166,Col167,Col168,Col169,Col170,Col171,Col172,Col173,Col174,Col175,Col176,Col177,Col178,Col179,Col180,Col181,Col182,Col183,Col184,Col185,Col186,Col187,Col188,Col189,Col190,Col191,Col192,Col193,Col194,Col195,Col196,Col197,Col198,Col199,Col200,Col201,Col202,Col203,Col204,Col205,Col206,Col207,Col208,Col209,Col210,Col211,Col212,Col213,Col214,Col215,Col216,Col217,Col218,Col219,Col220,Col221,Col222,Col223,Col224,Col225,Col226,Col227,Col228,Col229,Col230,Col231,Col232,Col233,Col234,Col235,Col236,Col237,Col238,Col239,Col240,Col241,Col242,Col243,Col244,Col245,Col246,Col247,Col248,Col249,Col250,Col251,Col252,Col253,Col254,Col255,Col256,Col257,Col258,Col259,Col260,Col261,Col262,Col263,Col264,Col265,Col266,Col267,Col268,Col269,Col270,Col271,Col272,Col273,Col274,Col275,Col276,Col277,Col278,Col279,Col280,Col281,Col282,Col283,Col284,Col285,Col286,Col287,Col288,Col289,Col290,Col291,Col292,Col293,Col294,Col295,Col296,Col297,Col298,Col299,Col300,Col301,Col302,Col303,Col304,Col305,Col306,Col307,Col308,Col309,Col310,Col311,Col312,Col313,Col314,Col315,Col316,Col317,Col318,Col319,Col320,Col321,Col322,Col323,Col324,Col325,Col326,Col327,Col328,Col329,Col330,Col331,Col332,Col333,Col334,Col335,Col336,Col337,Col338,Col339,Col340,Col341,Col342,Col343,Col344,Col345,Col346,Col347,Col348,Col349,Col350,Col351,Col352,Col353,Col354,Col355
Train_Nunq,316,"1,960",10,9,118,61,460,996,274,"1,138","1,059","1,083",635,36,"1,292",5,3,5,3,8,4,14,5,6,603,380,252,183,31,121,14,38,"1,744",17,12,"2,334",128,117,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2
Test_Nunq,239,"1,106",10,9,106,266,311,670,161,794,737,789,407,15,879,5,3,5,3,7,3,12,5,6,459,263,203,131,13,75,7,26,"1,060",15,12,"1,309",118,327,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,1,2,1,1,1,2,2,2,2,2,2,2,2,2,2,1,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,1,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,2,2,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2
Train_Nulls,0,0,0,0,0,0,0,0,0



CPU times: user 2.49 s, sys: 64.9 ms, total: 2.56 s
Wall time: 2.67 s


In [8]:
%%time 

class OptunaEnsembler:
    """
    This is the Optuna ensemble class-
    Source- https://www.kaggle.com/code/arunklenin/ps3e26-cirrhosis-survial-prediction-multiclass
    """
    
    def __init__(self):
        self.study        = None
        self.weights      = None
        self.random_state = CFG.state
        self.n_trials     = CFG.ntrials
        self.direction    = CFG.metric_obj
        
    def ScoreMetric(self, ytrue, ypred):
        """
        This is the metric function for the competition
        """
        return mse(ytrue, ypred, squared = False)

    def _objective(self, trial, y_true, y_preds):
        """
        This method defines the objective function for the ensemble
        """
        
        if isinstance(y_preds, pd.DataFrame) or isinstance(y_preds, np.ndarray):
            weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(y_preds.shape[-1])]
            axis = 1
        elif isinstance(y_preds, list):
            weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]
            axis = 0

        # Calculating the weighted prediction:-
        weighted_pred  = np.average(np.array(y_preds), axis = axis, weights = weights)
        score          = self.ScoreMetric(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        "This method fits the Optuna objective on the fold level data"
        
        optuna.logging.set_verbosity = optuna.logging.ERROR
        self.study = \
        optuna.create_study(sampler    = TPESampler(seed = self.random_state), 
                            pruner     = HyperbandPruner(),
                            study_name = "Ensemble", 
                            direction  = self.direction,
                           )
        
        obj = partial(self._objective, y_true = y_true, y_preds = y_preds)
        self.study.optimize(obj, n_trials = self.n_trials)
               
        if isinstance(y_preds, list):
            self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]
        else:
            self.weights = [self.study.best_params[f"weight{n}"] for n in range(y_preds.shape[-1])]
        clear_output()

    def predict(self, y_preds):
        "This method predicts using the fitted Optuna objective"
        
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        
        if isinstance(y_preds, list):
            weighted_pred = np.average(np.array(y_preds), axis=0, weights = self.weights)
        else:
            weighted_pred = np.average(np.array(y_preds), axis=1, weights = self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        """
        This method fits the Optuna objective on the fold data, then predicts the test set
        """
        self.fit(y_true, y_preds)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights
    
print()
collect()



CPU times: user 143 ms, sys: 1.11 ms, total: 144 ms
Wall time: 142 ms


In [9]:
%%time 

class MdlDeveloper(CFG):
    """
    This class implements the training pipeline elements-
    1. Initializes the Model predictions
    2. Trains and infers models
    3. Returns the OOF and model test set predictions
    """
    
    def __init__(self, Xtrain, ytrain, Xtest, sel_cols, **kwarg):
        """
        In this method, we initialize the below-
        1. Train-test data, selected columns
        2. Metric, custom scorer, model and cv object
        3. Output tables for score and predictions
        """
        
        self.Xtrain      = Xtrain
        self.ytrain      = ytrain
        self.y_grp       = ytrain
        self.Xtest       = Xtest
        self.sel_cols    = sel_cols
     
        self._DefineModels()
        self.cv          = self.all_cv[self.mdlcv_mthd]
        self.methods     = list(self.Mdl_Master.keys()) if self.test_req != "Y" else ["XGB1R", "LGBM1R"]
        self.OOF_Preds   = pd.DataFrame()
        self.Mdl_Preds   = pd.DataFrame()
        self.Scores      = pd.DataFrame(columns = self.methods + ["Ensemble"], 
                                        index = range(self.n_splits * self.n_repeats)
                                       ) 
        self.mdlscorer   = make_scorer(self.ScoreMetric, 
                                       greater_is_better = False,
                                       needs_proba       = True,
                                       needs_threshold   = False,
                                      )  
        
        PrintColor(f"\n---> Selected model options-")
        pprint(self.methods, depth = 1, width = 100, indent = 5)
              
    def _DefineModels(self):
        """
        This method initiliazes models for the analysis
        It also initializes the CV methods and class-weights that could be tuned going ahead.
        """
        
        # Commonly used CV strategies for later usage:-
        self.all_cv = \
        {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        }
               
        self.Mdl_Master = \
        {                      
         'XGB1R': XGBR(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'reg:squarederror',
                          'eval_metric'           : "rmse",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.25,
                          'learning_rate'         : 0.035,
                          'max_depth'             : 5,
                          'n_estimators'          : 1100,                         
                          'reg_alpha'             : 0.09,
                          'reg_lambda'            : 0.70,
                          'min_child_weight'      : 12,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
            
         'XGB2R': XGBR(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'reg:squarederror',
                          'eval_metric'           : "rmse",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.40,
                          'learning_rate'         : 0.02,
                          'max_depth'             : 6,
                          'n_estimators'          : 2500,                         
                          'reg_alpha'             : 0.12,
                          'reg_lambda'            : 0.8,
                          'min_child_weight'      : 15,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),

         'XGB3R': XGBR(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'reg:squarederror',
                          'eval_metric'           : "rmse",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.5,
                          'learning_rate'         : 0.04,
                          'max_depth'             : 7,
                          'n_estimators'          : 3000,                         
                          'reg_alpha'             : 0.2,
                          'reg_lambda'            : 0.6,
                          'min_child_weight'      : 16,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
            
         'XGB4R': XGBR(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'reg:squarederror',
                          'eval_metric'           : "rmse",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.80,
                          'learning_rate'         : 0.06,
                          'max_depth'             : 5,
                          'n_estimators'          : 2000,                         
                          'reg_alpha'             : 0.005,
                          'reg_lambda'            : 0.95,
                          'min_child_weight'      : 16,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
              
         'LGBM1R':LGBMR(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'regression_l2',
                           'metric'              : "rmse",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.56,
                           'subsample'           : 0.35,
                           'learning_rate'       : 0.03,
                           'max_depth'           : 6,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 180,
                           'reg_alpha'           : 0.14,
                           'reg_lambda'          : 0.85,
                           'verbosity'           : -1,
                          }
                       ),
            
         'LGBM2R':LGBMR(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'regression_l2',
                           'metric'              : "rmse",
                           'boosting_type'       : 'gbdt',
                           'data_sample_strategy': "goss",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.20,
                           'subsample'           : 0.25,
                           'learning_rate'       : 0.045,
                           'max_depth'           : 6,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 120, 
                           'reg_alpha'           : 0.15,
                           'reg_lambda'          : 0.90,
                           'verbosity'           : -1,
                          }
                       ),
            
         'LGBM3R':LGBMR(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'regression_l2',
                           'metric'              : "rmse",
                           'boosting_type'       : 'gbdt',
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.45,
                           'subsample'           : 0.45,
                           'learning_rate'       : 0.05,
                           'max_depth'           : 7,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 95, 
                           'reg_alpha'           : 0.05,
                           'reg_lambda'          : 0.95,
                           'verbosity'           : -1,
                          }
                       ), 
            
         'LGBM4R':LGBMR(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'regression_l2',
                           'metric'              : "rmse",
                           'boosting_type'       : 'gbdt',
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.55,
                           'subsample'           : 0.55,
                           'learning_rate'       : 0.035,
                           'max_depth'           : 6,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 76, 
                           'reg_alpha'           : 0.08,
                           'reg_lambda'          : 0.995,
                           'verbosity'           : -1,
                          }
                       ),
            
        "CB1R" :  CBR(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                         'objective'           : 'RMSE',
                         'eval_metric'         : "RMSE",
                         'bagging_temperature' : 0.1,
                         'colsample_bylevel'   : 0.88,
                         'iterations'          : 3000,
                         'learning_rate'       : 0.065,
                         'od_wait'             : 12,
                         'max_depth'           : 7,
                         'l2_leaf_reg'         : 1.75,
                         'min_data_in_leaf'    : 20,
                         'random_strength'     : 0.1, 
                         'max_bin'             : 100,
                         'verbose'             : 0,
                         'use_best_model'      : True,
                         'random_seed'         : self.state,
                        }
                     ),
            
        "CB2R" :  CBR(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                         'objective'           : 'RMSE',
                         'eval_metric'         : "RMSE",
                         'bagging_temperature' : 0.5,
                         'colsample_bylevel'   : 0.50,
                         'iterations'          : 2500,
                         'learning_rate'       : 0.0525,
                         'od_wait'             : 24,
                         'max_depth'           : 8,
                         'l2_leaf_reg'         : 1.235,
                         'min_data_in_leaf'    : 13,
                         'random_strength'     : 0.35, 
                         'max_bin'             : 160,
                         'verbose'             : 0,
                         'use_best_model'      : True,
                         'random_seed'         : self.state,
                        }
                     ),
        
        "HGBR" : HGBR(loss              = 'squared_error',
                      quantile          = None,
                      learning_rate     = 0.05,
                      max_iter          = 600,
                      max_depth         = 5,
                      min_samples_leaf  = 15,
                      l2_regularization = 1.5,
                      n_iter_no_change  = 50,
                      tol               = 1e-07,
                      verbose           = 0,
                      random_state      = self.state,
                     )  
        }
        return self
    
    def ScoreMetric(self, ytrue, ypred):
        """
        This is the metric function for the competition scoring
        """
        return mse(ytrue, ypred, squared = False)
       
    def PostProcessPred(self, ypred):
        """
        This is an optional post-processing method
        """
        return ypred
    
    def TrainMdl(self, test_preds_req: str = "Y"):
        """
        This method trains and infers from the model suite and returns the predictions and scores
        It optionally predicts the test set too, if desired by the user
        """

        # Initializing I-O:- 
        X,y, Xt    = self.Xtrain[self.sel_cols], self.ytrain.copy(deep = True), self.Xtest[self.sel_cols]
        cols_drop  = ['Source', "id", "Col36", "Col37"]
        ens        = OptunaEnsembler()
        
        self.FtreImp = pd.DataFrame(columns = self.methods, 
                                    index   = [c for c in self.sel_cols if c not in cols_drop]
                                   ).fillna(0)
        
        # Making CV folds:-        
        for fold_nb, (train_idx, dev_idx) in tqdm(enumerate(self.cv.split(X, self.y_grp))):
            Xtr  = X.iloc[train_idx].drop(columns = cols_drop, errors = 'ignore')  
            Xdev = X.iloc[dev_idx].drop(columns = cols_drop, errors = 'ignore')    
            ytr  = y.loc[y.index.isin(Xtr.index)]
            ydev = y.loc[y.index.isin(Xdev.index)]
                   
            # Initializing the OOF and test set predictions:-            
            oof_preds = pd.DataFrame(columns = self.methods, index = Xdev.index)
            mdl_preds = pd.DataFrame(columns = self.methods, index = Xt.index)
            
            PrintColor(f"\n{'=' * 5} FOLD {fold_nb + 1} {'=' * 5}\n")
            # Initializing models across methods:-
            for method in tqdm(self.methods):
                model = Pipeline(steps = [("M", self.Mdl_Master.get(method))]) 

                # Fitting the model:-          
                if "CB" in method:    
                    model.fit(Xtr, ytr, 
                              M__eval_set = [(Xdev, ydev)], 
                              M__verbose = 0,
                              M__early_stopping_rounds = CFG.nbrnd_erly_stp,
                             ) 

                elif "LGBM" in method:
                    model.fit(Xtr, ytr, 
                              M__eval_set = [(Xdev, ydev)], 
                              M__callbacks = [log_evaluation(0), 
                                              early_stopping(stopping_rounds = CFG.nbrnd_erly_stp, 
                                                             verbose = False,),
                                             ],
                             ) 

                elif "XGB" in method:
                     model.fit(Xtr, ytr, 
                               M__eval_set = [(Xdev, ydev)], 
                               M__verbose  = 0,
                              )            

                else: 
                    model.fit(Xtr, ytr)
                    
                # Collating feature importance:-
                try: 
                    self.FtreImp[method] += model["M"].feature_importances_
                except: 
                    pass
                    
                # Collecting predictions and scores and post-processing OOF based on model method:-
                dev_preds    = model.predict(Xdev)
                train_preds  = model.predict(Xtr) 
                tr_score     = self.ScoreMetric(ytr.values.flatten(),train_preds)
                score        = self.ScoreMetric(ydev.values.flatten(),dev_preds)

                PrintColor(f"Score:- OOF = {score:.5f} | Train = {tr_score:.5f}| {method}", 
                           color = Fore.CYAN
                          )  
                oof_preds[method] = dev_preds

                # Integrating the predictions and scores:-               
                self.Scores.at[fold_nb, method] = np.round(score, decimals= 6)
                
                if test_preds_req == "Y": 
                    mdl_preds[method] = model.predict(Xt.drop(columns = cols_drop, errors = "ignore"))
                
            try:
                del dev_preds, train_preds, tr_score, score, clb_score, clb_tr_preds, clb_dev_preds
            except:
                pass
                
            # Ensembling the predictions:-
            oof_preds["Ensemble"]          = ens.fit_predict(ydev, oof_preds[self.methods])
            clb_score                      = self.ScoreMetric(ydev, oof_preds["Ensemble"].values)
            self.OOF_Preds                 = pd.concat([self.OOF_Preds, oof_preds], axis = 0, ignore_index = False)
            self.Scores.at[fold_nb, "Ensemble"] = np.round(clb_score,6)
            
            if test_preds_req == "Y": 
                mdl_preds["Ensemble"] = ens.predict(mdl_preds[self.methods])
                self.Mdl_Preds        = pd.concat([self.Mdl_Preds, mdl_preds], axis = 0, ignore_index = False)
                
        # Averaging the predictions after all folds:-       
        self.OOF_Preds = self.OOF_Preds.groupby(level = 0).mean()
        if test_preds_req == "Y": 
            self.Mdl_Preds = self.Mdl_Preds[self.methods + ["Ensemble"]].groupby(level=0).mean()
            
        return self.OOF_Preds, self.Mdl_Preds, self.Scores
    
    def DisplayAdjTbl(self, *args):
        """
        This function displays pandas tables in an adjacent manner, sourced from the below link-
        https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
        """

        html_str = ''
        for df in args:
            html_str += df.to_html()
        display_html(html_str.replace('table','table style="display:inline"'),raw=True)
         
    def DisplayScores(self):
        "This method displays the scores and their means"
        
        PrintColor(f"\n---> OOF score across all methods and folds\n",color = Fore.LIGHTMAGENTA_EX)
        display(self.Scores.style.format(precision = 5).\
                background_gradient(cmap = "Pastel2", subset = self.methods).\
                background_gradient(cmap = "mako", subset = ["Ensemble"]).\
                set_caption(f"\nOOF scores across methods and folds\n")
               )
        
        PrintColor(f"\n---> Mean OOF score across all methods and folds\n",
                   color = Fore.RED
                  )
        
        display(self.Scores.mean().to_frame().\
                transpose().\
                style.format(precision = 5).\
                background_gradient(cmap = "terrain", subset = self.methods + ["Ensemble"], axis=1)
               )
        print()
      
    def MakePseudoLbl(self, up_cutoff: float, low_cutoff: float, **kwarg):
        """
        This method makes pseudo-labels using confident test set predictions to add to the training data
        """
        
        # Locating confident test-set predictions:-
        df = \
        self.Mdl_Preds.loc[(self.Mdl_Preds.Ensemble >= up_cutoff) | (self.Mdl_Preds.Ensemble <= low_cutoff), 
                           "Ensemble"
                          ]
        PrintColor(f"---> Pseudo Label additions from test set = {df.shape[0]:,.0f}", color = Fore.RED)
                
        #  Integrating new Xtrain and ytrain based on pseudo-labels:- 
        new_ytrain       = pd.concat([self.ytrain, df], axis=0, ignore_index = True)
        new_ytrain.index = range(len(new_ytrain))
        new_Xtrain       = pd.concat([self.Xtrain, self.Xtest.loc[df.index]], axis=0, ignore_index = True)
        new_Xtrain.index = range(len(new_Xtrain))
        
        #  Verifying the additions:-
        PrintColor(f"---> Revised train set shapes after pseudo labels = {new_Xtrain.shape} {new_ytrain.shape}")
        return new_Xtrain, new_ytrain
    
    def MakeMLPlots(self):
        """
        This method makes plots for the ML models, including feature importance and calibration curves
        """
        
        fig, axes = plt.subplots(len(self.methods), 1, figsize = (30, len(self.methods) * 9), 
                                 sharex = False,
                                )
    
        for i, col in enumerate(self.methods):
            try: 
                ax = axes[i]
            except: 
                ax = axes[0]
                
            self.FtreImp[col].plot.bar(ax = ax, color = '#0073e6')
            ax.set_title(f"{col} Importances", **CFG.title_specs)
            ax.set(xlabel = '', ylabel = '')
            ax.legend('')

        plt.tight_layout()
        plt.show()
                   
print()
collect()


CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 141 ms


In [10]:
%%time

if CFG.ML == "Y":
    sel_cols = np.array(Xtrain.columns)
    PrintColor(f"\n---> Selected model columns")
    with np.printoptions(linewidth = 120):
        pprint(sel_cols)
        
print()
collect()


---> Selected model columns
array(['Col0', 'Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6', 'Col7', 'Col8', 'Col9', 'Col10', 'Col11', 'Col12',
       'Col13', 'Col14', 'Col15', 'Col16', 'Col17', 'Col18', 'Col19', 'Col20', 'Col21', 'Col22', 'Col23', 'Col24',
       'Col25', 'Col26', 'Col27', 'Col28', 'Col29', 'Col30', 'Col31', 'Col32', 'Col33', 'Col34', 'Col35', 'Col36',
       'Col37', 'Col38', 'Col39', 'Col40', 'Col41', 'Col42', 'Col43', 'Col44', 'Col45', 'Col46', 'Col47', 'Col48',
       'Col49', 'Col50', 'Col51', 'Col52', 'Col53', 'Col54', 'Col55', 'Col56', 'Col57', 'Col58', 'Col59', 'Col60',
       'Col61', 'Col62', 'Col63', 'Col64', 'Col65', 'Col66', 'Col67', 'Col68', 'Col69', 'Col70', 'Col71', 'Col72',
       'Col73', 'Col74', 'Col75', 'Col76', 'Col77', 'Col78', 'Col79', 'Col80', 'Col81', 'Col82', 'Col83', 'Col84',
       'Col85', 'Col86', 'Col87', 'Col88', 'Col89', 'Col90', 'Col91', 'Col92', 'Col93', 'Col94', 'Col95', 'Col96',
       'Col97', 'Col98', 'Col99', 'Col100', 'Col101'

In [11]:
%%time 

if CFG.ML == "Y":
    # Training the models with a CV analysis:-  
    md = MdlDeveloper(Xtrain, ytrain, Xtest, sel_cols = sel_cols)
    OOF_Preds, Mdl_Preds, Scores = md.TrainMdl(test_preds_req = "Y")
    
    PrintColor(f"\n{'=' * 20} ML MODELS TRAINING AND CV {'=' * 20}\n", color = Fore.MAGENTA)
    md.DisplayScores()
    collect()
    
print()
collect()



==================== ML MODELS TRAINING AND CV ====================


---> OOF score across all methods and folds



,XGB1R,XGB2R,XGB3R,XGB4R,LGBM1R,LGBM2R,LGBM3R,LGBM4R,CB1R,CB2R,HGBR,Ensemble
0,0.10279,0.10558,0.10874,0.11060,0.10584,0.10526,0.10767,0.10727,0.09487,0.10016,0.11006,0.09784
1,0.11483,0.11595,0.11810,0.11867,0.11594,0.12273,0.11666,0.11683,0.11438,0.11642,0.11945,0.11328
2,0.14237,0.14513,0.14539,0.14153,0.14424,0.14730,0.14474,0.14363,0.14328,0.14379,0.14777,0.14075
3,0.11623,0.11671,0.11745,0.11973,0.11825,0.11859,0.11743,0.11729,0.11337,0.11233,0.12178,0.11216
4,0.10475,0.10675,0.11002,0.11073,0.10858,0.10739,0.10787,0.10956,0.10361,0.10434,0.10946,0.10224



---> Mean OOF score across all methods and folds



,XGB1R,XGB2R,XGB3R,XGB4R,LGBM1R,LGBM2R,LGBM3R,LGBM4R,CB1R,CB2R,HGBR,Ensemble
0,0.11619,0.11802,0.11994,0.12025,0.11857,0.12025,0.11888,0.11892,0.11390,0.11541,0.12170,0.11325




CPU times: user 8min 38s, sys: 32.6 s, total: 9min 10s
Wall time: 6min 1s


In [12]:
%%time 

if CFG.ML == "Y" and CFG.pseudo_lbl_req == "Y":
    Xtrain, ytrain = md.MakePseudoLbl(up_cutoff = CFG.pseudolbl_up, low_cutoff = CFG.pseudolbl_low)
    
    # Re-initializing the developer class with the new training set:-  
    md = MdlDeveloper(Xtrain, ytrain, Xtest, sel_cols = sel_cols)
    OOF_Preds, Mdl_Preds, Scores = md.TrainMdl(test_preds_req = "Y")
    
    PrintColor(f"\n{'=' * 20} ML MODELS TRAINING AND CV AFTER PSEUDO-LABELS {'=' * 20}\n", 
               color = Fore.MAGENTA
              )
    md.DisplayScores()
    collect()    
    
else:
    PrintColor(f"---> Pseudo Labels are not needed", Fore.RED)

print()
collect()

---> Pseudo Labels are not needed

CPU times: user 145 ms, sys: 945 µs, total: 145 ms
Wall time: 144 ms


In [13]:
%%time 

# Saving the datasets:-
if CFG.ML == "Y":
    sub_fl = pp.sub_fl.copy()
    sub_fl[CFG.target] = Mdl_Preds["Ensemble"].values.flatten()
    sub_fl[CFG.target] = np.expm1(sub_fl[CFG.target])
      
    # Blending with good public notebooks:-
    sub1 = pd.read_csv(f"/kaggle/input/xgboost-bayes-opt-with-tidymodels/submission.csv")
    sub2 = pd.read_csv(f"/kaggle/input/house-regression-beginner-catboost-top-2/submission.csv")
    sub3 = pd.read_csv(f"/kaggle/input/house-prices-advanced-regression-technique/result.csv")

    sub_fl[CFG.target] = \
    np.average(np.c_[sub_fl[CFG.target].values, 
                     sub1[CFG.target].values, 
                     sub2[CFG.target].values, 
                     sub3[CFG.target].values
                    ], axis = 1, weights = CFG.blend_wgt,
              )

    sub_fl.to_csv(f"Submission_V{CFG.version_nb}.csv", index = None)
    print()
    
    OOF_Preds.to_csv(f"OOF_Preds_V{CFG.version_nb}.csv")
    Mdl_Preds.to_csv(f"Mdl_Preds_V{CFG.version_nb}.csv")
    
    PrintColor(f"\nFinal submission file\n")
    display(sub_fl.head(10).style.format(precision = 2))
    
collect()
print() 



Final submission file



,Id,SalePrice
0,1461,124901.80
1,1462,169801.91
2,1463,187782.01
3,1464,193189.23
4,1465,185395.62
5,1466,175936.32
6,1467,180083.62
7,1468,171107.90
8,1469,183702.03
9,1470,122713.41



CPU times: user 219 ms, sys: 3.8 ms, total: 223 ms
Wall time: 243 ms
